In [202]:
# Importando as Bibliotecas Necessárias
import pandas as pd
import numpy as np

# Modelos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn_lvq import GlvqModel

# Validação e métricas
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, roc_curve
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

# Bibliotecas adicionais
import optuna
from optuna.integration.mlflow import MLflowCallback
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [203]:
# Configurando o MLflow
mlflow.set_experiment("Modelagem de Classificação")

<Experiment: artifact_location='file:///c:/Users/PC/Documents/GitHub/diabetes/mlruns/490920415060799094', creation_time=1732409763939, experiment_id='490920415060799094', last_update_time=1732409763939, lifecycle_stage='active', name='Modelagem de Classificação', tags={}>

In [204]:
# Carregando os conjuntos de dados
train_data = pd.read_csv('train_data.csv')
validation_data = pd.read_csv('validation_data.csv')
test_data = pd.read_csv('test_data.csv')

In [205]:
# def convert_bool_to_int(df):
#     bool_cols = df.select_dtypes(include=['bool']).columns
#     df[bool_cols] = df[bool_cols].astype('int64')
#     return df

# # Aplicando a conversão nos conjuntos de treinamento e validação
# train_data = convert_bool_to_int(train_data)
# validation_data = convert_bool_to_int(validation_data)

In [206]:
# Separando features e target no conjunto de treinamento
X_train = train_data.drop('class', axis=1)
y_train = train_data['class']

# Separando features e target no conjunto de validação
X_val = validation_data.drop('class', axis=1)
y_val = validation_data['class']

# Conjunto de teste
X_test = test_data.drop('class', axis=1)
y_test = test_data['class']

In [207]:
# Colunas que estão tipo 'object' em X_test
cols_to_convert = ['race', 'gender', 'age', 'weight', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide.metformin', 'glipizide.metformin', 'glimepiride.pioglitazone', 'metformin.rosiglitazone', 'metformin.pioglitazone', 'change', 'diabetesMed']

# Convertendo essas colunas para 'category' em X_test
for col in cols_to_convert:
    if col in X_test.columns:
        X_test[col] = X_test[col].astype('category')

In [208]:
# Definindo uma Função de Avaliação
def evaluate_model(model, X_train, y_train, X_val, y_val, model_name):
    # Treinando e avaliando o modelo com os dados originais
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    y_true = y_val
    X_val_used = X_val

    # Calculando as métricas
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_val_used)
        if y_prob.ndim == 1 or y_prob.shape[1] == 2:
            y_prob = y_prob[:, -1]  # Para classes binárias
            roc_auc = roc_auc_score(y_true, y_prob)
        else:
            roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovo', average='weighted')
    else:
        roc_auc = None
    return acc, f1, recall, roc_auc


In [209]:
# Definindo os Modelos
models = {
    'K-NN': KNeighborsClassifier(),
    'LVQ': NearestCentroid(),
    'Árvore de Decisão': DecisionTreeClassifier(),
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(),
    'Rede Neural MLP': MLPClassifier(max_iter=500),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'LightGBM': LGBMClassifier()
}

In [210]:
# Comitê de Redes Neurais Artificiais
# Definindo múltiplas redes neurais
nn1 = MLPClassifier(hidden_layer_sizes=(50,), activation='relu', solver='adam', max_iter=500, random_state=1)
nn2 = MLPClassifier(hidden_layer_sizes=(100,), activation='tanh', solver='sgd', max_iter=500, random_state=2)
nn3 = MLPClassifier(hidden_layer_sizes=(50, 50), activation='relu', solver='adam', max_iter=500, random_state=3)

# Criando o Comitê
committee_nn = VotingClassifier(estimators=[
    ('nn1', nn1),
    ('nn2', nn2),
    ('nn3', nn3)
], voting='soft')

models['Comitê de Redes Neurais Artificiais'] = committee_nn


In [211]:
# Comitê Heterogêneo (Stacking)
# Modelos base
estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(random_state=42)),
    ('knn', KNeighborsClassifier())
]

# Modelo meta
from sklearn.linear_model import LogisticRegression
final_estimator = LogisticRegression()

# Criando o StackingClassifier
stacking_model = StackingClassifier(estimators=estimators, final_estimator=final_estimator, cv=5)

models['Comitê Heterogêneo (Stacking)'] = stacking_model

In [212]:
# Executando os Modelos Básicos e Registrando no MLflow
initial_results = []

for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        acc, f1, recall, roc_auc = evaluate_model(model, X_train, y_train, X_val, y_val, model_name)
        mlflow.log_param("model_type", model_name)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("recall", recall)
        if roc_auc is not None:
            mlflow.log_metric("roc_auc", roc_auc)
        # Salvar o modelo
        mlflow.sklearn.log_model(model, model_name)
        print(f"{model_name} - Acurácia: {acc}, F1-Score: {f1}, Recall: {recall}, ROC AUC: {roc_auc}")
        initial_results.append({
            'Modelo': model_name,
            'Acurácia': acc,
            'F1-Score': f1,
            'Recall': recall,
            'ROC AUC': roc_auc
        })

# Converte os resultados iniciais em DataFrame
initial_results_df = pd.DataFrame(initial_results)

# Exibindo os resultados iniciais
print(initial_results_df)

2024/11/24 01:28:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


K-NN - Acurácia: 0.8749487014129097, F1-Score: 0.8305980818922424, Recall: 0.8749487014129097, ROC AUC: 0.5054859295809839


2024/11/24 01:28:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


LVQ - Acurácia: 0.5108166735064783, F1-Score: 0.5965374024166652, Recall: 0.5108166735064783, ROC AUC: None


2024/11/24 01:28:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

LightGBM - Acurácia: 0.8870258544878935, F1-Score: 0.835878963507557, Recall: 0.8870258544878935, ROC AUC: 0.6652721930265553


2024/11/24 01:32:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Comitê de Redes Neurais Artificiais - Acurácia: 0.8867327197045202, F1-Score: 0.8345094391946535, Recall: 0.8867327197045202, ROC AUC: 0.6561677114848266


2024/11/24 01:42:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Comitê Heterogêneo (Stacking) - Acurácia: 0.8864982118778214, F1-Score: 0.8347251165452486, Recall: 0.8864982118778214, ROC AUC: 0.642877957724016
                                Modelo  Acurácia  F1-Score    Recall   ROC AUC
0                                 K-NN  0.874949  0.830598  0.874949  0.505486
1                                  LVQ  0.510817  0.596537  0.510817       NaN
2                    Árvore de Decisão  0.797209  0.803571  0.797209  0.528575
3                                  SVM  0.886967  0.833836  0.886967       NaN
4                        Random Forest  0.886322  0.834078  0.886322  0.641122
5                      Rede Neural MLP  0.886205  0.835017  0.886205  0.652663
6                              XGBoost  0.885443  0.836868  0.885443  0.645512
7                             LightGBM  0.887026  0.835879  0.887026  0.665272
8  Comitê de Redes Neurais Artificiais  0.886733  0.834509  0.886733  0.656168
9        Comitê Heterogêneo (Stacking)  0.886498  0.834725  0.8

In [213]:
# Busca de Hiperparâmetros com Optuna
# Configurando o Callback do MLflow para o Optuna
mlflc = MLflowCallback(tracking_uri=mlflow.get_tracking_uri(), metric_name='accuracy')


C:\Users\PC\AppData\Local\Temp\ipykernel_9704\2387988113.py:3: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(tracking_uri=mlflow.get_tracking_uri(), metric_name='accuracy')


In [214]:
def build_classifier(model_name, params):
    if model_name == 'K-NN':
        classifier = KNeighborsClassifier(
            n_neighbors=params['n_neighbors'],
            weights=params['weights'],
            algorithm=params['algorithm']
        )
    elif model_name == 'LVQ':
        classifier = NearestCentroid(
            metric=params['metric'],
            shrink_threshold=params['shrink_threshold']
        )
    elif model_name == 'Árvore de Decisão':
        classifier = DecisionTreeClassifier(
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split'],
            criterion=params['criterion']
        )
    elif model_name == 'SVM':
        classifier = SVC(
            C=params['C'],
            kernel=params['kernel'],
            gamma=params['gamma']
        )
    elif model_name == 'Random Forest':
        classifier = RandomForestClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split']
        )
    elif model_name == 'Rede Neural MLP':
        classifier = MLPClassifier(
            hidden_layer_sizes=params['hidden_layer_sizes'],
            activation=params['activation'],
            solver=params['solver'],
            alpha=params['alpha'],
            max_iter=500
        )
    elif model_name == 'Comitê de Redes Neurais Artificiais':
        nn1 = MLPClassifier(
            hidden_layer_sizes=params['hidden_layer_sizes'],
            activation=params['activation'],
            solver=params['solver'],
            max_iter=500,
            random_state=1
        )
        nn2 = MLPClassifier(
            hidden_layer_sizes=params['hidden_layer_sizes'],
            activation=params['activation'],
            solver=params['solver'],
            max_iter=500,
            random_state=2
        )
        nn3 = MLPClassifier(
            hidden_layer_sizes=params['hidden_layer_sizes'],
            activation=params['activation'],
            solver=params['solver'],
            max_iter=500,
            random_state=3
        )
        classifier = VotingClassifier(
            estimators=[
                ('nn1', nn1),
                ('nn2', nn2),
                ('nn3', nn3)
            ],
            voting='soft'
        )
    elif model_name == 'Comitê Heterogêneo (Stacking)':
        final_estimator = LogisticRegression(
            C=params['C'],
            penalty=params['penalty'],
            solver=params['solver']
        )
        estimators = [
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
            ('svm', SVC(random_state=42)),
            ('knn', KNeighborsClassifier())
        ]
        classifier = StackingClassifier(
            estimators=estimators,
            final_estimator=final_estimator,
            cv=5
        )
    elif model_name == 'XGBoost':
        classifier = XGBClassifier(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            learning_rate=params['learning_rate'],
            subsample=params['subsample'],
            use_label_encoder=False,
            eval_metric='logloss'
        )
    elif model_name == 'LightGBM':
        classifier = LGBMClassifier(
            n_estimators=params['n_estimators'],
            num_leaves=params['num_leaves'],
            learning_rate=params['learning_rate'],
            subsample=params['subsample']
        )
    else:
        raise ValueError(f"Modelo {model_name} não reconhecido.")
    return classifier


In [215]:
def objective_factory(model_name):
    def objective(trial):
        params = {}
        # Definir os hiperparâmetros para cada modelo
        if model_name == 'K-NN':
            params['n_neighbors'] = trial.suggest_int('n_neighbors', 1, 30)
            params['weights'] = trial.suggest_categorical('weights', ['uniform', 'distance'])
            params['algorithm'] = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])
        
        elif model_name == 'LVQ':
            params['metric'] = trial.suggest_categorical('metric', ['euclidean', 'manhattan'])
            use_shrinkage = trial.suggest_categorical('use_shrinkage', [True, False])
            params['shrink_threshold'] = trial.suggest_float('shrink_threshold', 0.0, 1.0) if use_shrinkage else None
        
        elif model_name == 'Árvore de Decisão':
            params['max_depth'] = trial.suggest_int('max_depth', 1, 20)
            params['min_samples_split'] = trial.suggest_int('min_samples_split', 2, 10)
            params['criterion'] = trial.suggest_categorical('criterion', ['gini', 'entropy'])
        
        elif model_name == 'SVM':
            params['C'] = trial.suggest_float('C', 0.1, 10.0, log=True)
            params['kernel'] = trial.suggest_categorical('kernel', ['linear', 'rbf'])
            params['gamma'] = trial.suggest_categorical('gamma', ['scale', 'auto'])
        
        elif model_name == 'Random Forest':
            params['n_estimators'] = trial.suggest_int('n_estimators', 50, 300)
            params['max_depth'] = trial.suggest_int('max_depth', 2, 20)
            params['min_samples_split'] = trial.suggest_int('min_samples_split', 2, 10)
        
        elif model_name == 'Rede Neural MLP':
            params['hidden_layer_sizes'] = trial.suggest_categorical('hidden_layer_sizes', [(50,), (100,), (50,50)])
            params['activation'] = trial.suggest_categorical('activation', ['tanh', 'relu'])
            params['solver'] = trial.suggest_categorical('solver', ['sgd', 'adam'])
            params['alpha'] = trial.suggest_float('alpha', 1e-5, 1e-1, log=True)
        
        elif model_name == 'Comitê de Redes Neurais Artificiais':
            params['hidden_layer_sizes'] = trial.suggest_categorical('hidden_layer_sizes', [(50,), (100,), (50,50)])
            params['activation'] = trial.suggest_categorical('activation', ['tanh', 'relu'])
            params['solver'] = trial.suggest_categorical('solver', ['sgd', 'adam'])
        
        elif model_name == 'Comitê Heterogêneo (Stacking)':
            params['C'] = trial.suggest_float('C', 0.1, 10.0, log=True)
            params['penalty'] = trial.suggest_categorical('penalty', ['l2'])
            params['solver'] = trial.suggest_categorical('solver', ['lbfgs'])
        
        elif model_name == 'XGBoost':
            params['n_estimators'] = trial.suggest_int('n_estimators', 50, 300)
            params['max_depth'] = trial.suggest_int('max_depth', 3, 15)
            params['learning_rate'] = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
            params['subsample'] = trial.suggest_float('subsample', 0.5, 1.0)
        
        elif model_name == 'LightGBM':
            params['n_estimators'] = trial.suggest_int('n_estimators', 50, 300)
            params['num_leaves'] = trial.suggest_int('num_leaves', 31, 150)
            params['learning_rate'] = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
            params['subsample'] = trial.suggest_float('subsample', 0.5, 1.0)
        
        else:
            raise ValueError(f"Modelo {model_name} não reconhecido.")
        
        classifier = build_classifier(model_name, params)
        
        acc = cross_val_score(classifier, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1).mean()
        return acc
    return objective


In [217]:
# Executando as Otimizações com Optuna
n_trials = 5  # Número de iterações

In [218]:
# Dicionário para armazenar os melhores modelos e seus scores
best_models = {}
best_params = {}
cv_results = {}


In [ ]:
for model_name in models.keys():
    print(f"Otimização para o modelo: {model_name}")
    study = optuna.create_study(direction='maximize', study_name='Modelagem', sampler=optuna.samplers.TPESampler(seed=123))
    study.optimize(objective_factory(model_name), n_trials=n_trials, callbacks=[mlflc])
    
    # Armazenando os melhores hiperparâmetros
    best_params[model_name] = study.best_params
    
    # Criando o classificador com os melhores hiperparâmetros
    classifier = build_classifier(model_name, best_params[model_name])
    
    # Armazenando o classificador
    best_models[model_name] = classifier
    
    # Armazenando os scores dos folds
    acc_scores = cross_val_score(classifier, X_train, y_train, cv=5, scoring='accuracy')
    cv_results[model_name] = acc_scores


[I 2024-11-24 01:42:11,191] A new study created in memory with name: Modelagem


Otimização para o modelo: K-NN


[I 2024-11-24 01:42:18,874] Trial 0 finished with value: 0.8869362571910037 and parameters: {'n_neighbors': 21, 'weights': 'uniform', 'algorithm': 'brute'}. Best is trial 0 with value: 0.8869362571910037.
[I 2024-11-24 01:43:14,945] Trial 1 finished with value: 0.8869362571910037 and parameters: {'n_neighbors': 21, 'weights': 'uniform', 'algorithm': 'ball_tree'}. Best is trial 0 with value: 0.8869362571910037.
[I 2024-11-24 01:43:22,448] Trial 2 finished with value: 0.8867750356378183 and parameters: {'n_neighbors': 12, 'weights': 'uniform', 'algorithm': 'brute'}. Best is trial 0 with value: 0.8869362571910037.
[I 2024-11-24 01:43:29,664] Trial 3 finished with value: 0.8869362571910037 and parameters: {'n_neighbors': 26, 'weights': 'uniform', 'algorithm': 'auto'}. Best is trial 0 with value: 0.8869362571910037.
[I 2024-11-24 01:43:39,327] Trial 4 finished with value: 0.8856171356265025 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'algorithm': 'kd_tree'}. Best is trial 0 wit

Otimização para o modelo: LVQ


[I 2024-11-24 01:43:45,917] Trial 0 finished with value: 0.5102085562235421 and parameters: {'metric': 'euclidean', 'use_shrinkage': False}. Best is trial 0 with value: 0.5102085562235421.
[I 2024-11-24 01:43:46,415] Trial 1 finished with value: 0.5095783059121426 and parameters: {'metric': 'euclidean', 'use_shrinkage': True, 'shrink_threshold': 0.48093190148436094}. Best is trial 0 with value: 0.5102085562235421.
[I 2024-11-24 01:43:46,900] Trial 2 finished with value: 0.5101059577602903 and parameters: {'metric': 'euclidean', 'use_shrinkage': True, 'shrink_threshold': 0.05967789660956835}. Best is trial 0 with value: 0.5102085562235421.
[I 2024-11-24 01:43:47,446] Trial 3 finished with value: 0.5331026784311701 and parameters: {'metric': 'manhattan', 'use_shrinkage': True, 'shrink_threshold': 0.5315513738418384}. Best is trial 3 with value: 0.5331026784311701.
[I 2024-11-24 01:43:48,005] Trial 4 finished with value: 0.5327948798190674 and parameters: {'metric': 'manhattan', 'use_shri

Otimização para o modelo: Árvore de Decisão


[I 2024-11-24 01:43:49,646] Trial 0 finished with value: 0.8718688846226204 and parameters: {'max_depth': 14, 'min_samples_split': 4, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8718688846226204.
[I 2024-11-24 01:43:50,598] Trial 1 finished with value: 0.8683512199565122 and parameters: {'max_depth': 15, 'min_samples_split': 5, 'criterion': 'gini'}. Best is trial 0 with value: 0.8718688846226204.
[I 2024-11-24 01:43:51,375] Trial 2 finished with value: 0.8813959391012883 and parameters: {'max_depth': 10, 'min_samples_split': 5, 'criterion': 'entropy'}. Best is trial 2 with value: 0.8813959391012883.
[I 2024-11-24 01:43:52,098] Trial 3 finished with value: 0.8827297201976781 and parameters: {'max_depth': 9, 'min_samples_split': 2, 'criterion': 'entropy'}. Best is trial 3 with value: 0.8827297201976781.
[I 2024-11-24 01:43:52,577] Trial 4 finished with value: 0.8868189938252844 and parameters: {'max_depth': 4, 'min_samples_split': 3, 'criterion': 'entropy'}. Best is trial 4 wi

Otimização para o modelo: SVM


In [ ]:
# Avaliação dos Melhores Modelos e Registro no MLflow
results = []

for model_name, model in best_models.items():
    with mlflow.start_run(run_name=f"{model_name} - Optuna HPO"):
        acc, f1, recall, roc_auc = evaluate_model(model, X_train, y_train, X_val, y_val)
        mlflow.log_params(best_params[model_name])
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("recall", recall)
        if roc_auc is not None:
            mlflow.log_metric("roc_auc", roc_auc)
        # Salvar o modelo
        mlflow.sklearn.log_model(model, model_name)
        print(f"{model_name} com Optuna HPO - Acurácia: {acc}, F1-Score: {f1}, Recall: {recall}, ROC AUC: {roc_auc}")
        results.append({
            'Modelo': model_name,
            'Acurácia': acc,
            'F1-Score': f1,
            'Recall': recall,
            'ROC AUC': roc_auc
        })

# Converte os resultados em DataFrame
results_df = pd.DataFrame(results)

In [ ]:
# Selecionando o Melhor Modelo automaticamente com base na Acurácia
best_model_name = results_df.loc[results_df['Acurácia'].idxmax(), 'Modelo']
best_model = best_models[best_model_name]
print(f"O melhor modelo é: {best_model_name}")

In [ ]:
# Treinando o Melhor Modelo no Conjunto Combinado e Avaliando no Conjunto de Teste
# Combinando os conjuntos de treinamento e validação
X_combined = pd.concat([X_train, X_val])
y_combined = np.concatenate([y_train, y_val])

# Treinando o melhor modelo
best_model.fit(X_combined, y_combined)

# Avaliando no conjunto de teste
y_pred_test = best_model.predict(X_test)
acc_test = accuracy_score(y_test, y_pred_test)
f1_test = f1_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
if hasattr(best_model, "predict_proba"):
    y_prob_test = best_model.predict_proba(X_test)
    if y_prob_test.ndim == 1 or y_prob_test.shape[1] == 2:
        y_prob_test = y_prob_test[:, -1]  # Para classes binárias
        roc_auc_test = roc_auc_score(y_test, y_prob_test)
    else:
        roc_auc_test = roc_auc_score(y_test, y_prob_test, multi_class='ovo', average='weighted')
else:
    roc_auc_test = None

print(f"Desempenho no conjunto de teste - Acurácia: {acc_test}, F1-Score: {f1_test}, Recall: {recall_test}, ROC AUC: {roc_auc_test}")


In [ ]:
# Registrando no MLflow
with mlflow.start_run(run_name="Melhor Modelo - Teste"):
    mlflow.log_param("model_type", best_model_name)
    mlflow.log_metric("accuracy_test", acc_test)
    mlflow.log_metric("f1_score_test", f1_test)
    mlflow.log_metric("recall_test", recall_test)
    if roc_auc_test is not None:
        mlflow.log_metric("roc_auc_test", roc_auc_test)
    mlflow.sklearn.log_model(best_model, "Melhor_Modelo")


In [ ]:
# Visualizações Adicionais
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Matriz de Confusão
conf_matrix = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(8,6))
plt.title("Matriz de Confusão - Conjunto de Teste")
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.show()

In [ ]:
# Curva ROC
if roc_auc_test is not None and len(np.unique(y_test)) == 2:
    fpr, tpr, thresholds = roc_curve(y_test, y_prob_test)
    plt.figure(figsize=(8,6))
    plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc_test:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')  # Linha diagonal
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Curva ROC - Conjunto de Teste')
    plt.legend(loc='lower right')
    plt.show()
elif roc_auc_test is not None:
    print("Curva ROC para problemas multiclasse não está implementada neste código.")


In [ ]:
# Comparação dos Modelos
# Já temos o DataFrame results_df com as métricas dos modelos
print(results_df)

In [ ]:
# Plotando as métricas
results_df_melted = results_df.melt(id_vars='Modelo', value_vars=['Acurácia', 'F1-Score', 'Recall', 'ROC AUC'], var_name='Métrica', value_name='Valor')

plt.figure(figsize=(12,8))
sns.barplot(x='Modelo', y='Valor', hue='Métrica', data=results_df_melted)
plt.xticks(rotation=45)
plt.title('Comparação das Métricas dos Modelos')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Implementação da Metodologia de Janez Demsar
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp

# Usando os scores de cross-validation armazenados em cv_results
scores_df = pd.DataFrame(cv_results)

# Teste de Friedman
stat, p = friedmanchisquare(*[scores_df[model] for model in scores_df.columns])
print(f'Estatística: {stat}, p-valor: {p}')

# Se p-valor < 0.05, há diferença significativa
if p < 0.05:
    print('Diferença significativa entre os modelos. Realizando teste de Nemenyi.')
    nemenyi = sp.posthoc_nemenyi_friedman(scores_df)
    print(nemenyi)
else:
    print('Não há diferença significativa entre os modelos.')